In [5]:
!pwd

/home/micha/0_Dev/HACKATHON/AI_Cybersecurity_IDS_PoC/analysis/RF


In [6]:
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler    
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

In [7]:
#df = pd.read_pickle('../data/conn_202011241955.pickle')
df_test = pd.read_pickle('../data/unsw-small.pickle')

In [8]:
#df_test.rename(columns={"spkts": "orig_pkts", "dpkts": "resp_pkts", "sbytes": "orig_ip_bytes", "dbytes": "resp_ip_bytes"}, errors="raise", inplace=True)

In [11]:
df_new = df_test

No.,Name,Type ,Description
* 7,dur,Float,Record total duration

* 8,sbytes,Integer,Source to destination transaction bytes (orig_ip_bytes)
* 9,dbytes,Integer,Destination to source transaction bytes (resp_ip_bytes)

* 15,Sload,Float,Source bits per second ((orig_ip_bytes/8) * orig_pkts)
* 16,Dload,Float,Destination bits per second ((resp_ip_bytes/8) * resp_pkts)

* 17,Spkts,integer,Source to destination packet count (orig_pkts)
* 18,Dpkts,integer,Destination to source packet count (resp_pkts)

* 23,smeansz,integer,Mean of the ?ow packet size transmitted by the src 
* 24,dmeansz,integer,Mean of the ?ow packet size transmitted by the dst

* 48,attack_cat,nominal,"The name of each attack category. In this data set , nine categories e.g. Fuzzers, Analysis, Backdoors, DoS Exploits, Generic, Reconnaissance, Shellcode and Worms"
* 49,Label,binary,0 for normal and 1 for attack records

In [7]:
#del df_new['Unnamed: 0']
#del df_new['id']

In [10]:
"""
del df_new['ct_dst_sport_ltm']
del df_new['ct_dst_src_ltm']
del df_new['is_ftp_login']
del df_new['ct_ftp_cmd']
del df_new['ct_flw_http_mthd']
del df_new['ct_src_ltm']
del df_new['ct_srv_dst']
del df_new['is_sm_ips_ports']
del df_new['trans_depth']
del df_new['response_body_len']
del df_new['ct_srv_src']
del df_new['ct_state_ttl']
del df_new['ct_dst_ltm']
del df_new['ct_src_dport_ltm']	 	 	 
del df_new['proto']
del df_new['service']
del df_new['state']
del df_new['dwin']
del df_new['tcprtt']
del df_new['synack']
del df_new['ackdat']
del df_new['sloss']
del df_new['dloss']
del df_new['stcpb']
del df_new['dtcpb']
del df_new['smean'] #orig_ip_bytes / orig_pkts
del df_new['dmean'] #resp_ip_bytes / resp_pkts
del df_new['sttl']
del df_new['dttl']
del df_new['sload']
del df_new['dload']
del df_new['sinpkt']
del df_new['dinpkt']
del df_new['sjit']
del df_new['djit']
del df_new['swin']
"""


"\ndel df_new['ct_dst_sport_ltm']\ndel df_new['ct_dst_src_ltm']\ndel df_new['is_ftp_login']\ndel df_new['ct_ftp_cmd']\ndel df_new['ct_flw_http_mthd']\ndel df_new['ct_src_ltm']\ndel df_new['ct_srv_dst']\ndel df_new['is_sm_ips_ports']\ndel df_new['trans_depth']\ndel df_new['response_body_len']\ndel df_new['ct_srv_src']\ndel df_new['ct_state_ttl']\ndel df_new['ct_dst_ltm']\ndel df_new['ct_src_dport_ltm']\t \t \t \ndel df_new['proto']\ndel df_new['service']\ndel df_new['state']\ndel df_new['dwin']\ndel df_new['tcprtt']\ndel df_new['synack']\ndel df_new['ackdat']\ndel df_new['sloss']\ndel df_new['dloss']\ndel df_new['stcpb']\ndel df_new['dtcpb']\ndel df_new['smean'] #orig_ip_bytes / orig_pkts\ndel df_new['dmean'] #resp_ip_bytes / resp_pkts\ndel df_new['sttl']\ndel df_new['dttl']\ndel df_new['sload']\ndel df_new['dload']\ndel df_new['sinpkt']\ndel df_new['dinpkt']\ndel df_new['sjit']\ndel df_new['djit']\ndel df_new['swin']\n"

In [12]:
df_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 257673 entries, 0 to 257672
Data columns (total 6 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   duration       257673 non-null  float64
 1   orig_pkts      257673 non-null  int64  
 2   orig_ip_bytes  257673 non-null  int64  
 3   resp_pkts      257673 non-null  int64  
 4   resp_ip_bytes  257673 non-null  int64  
 5   label          257673 non-null  int64  
dtypes: float64(1), int64(5)
memory usage: 11.8 MB


In [13]:
df_new

,duration,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,label
0,0.121478,6,258,4,172,0
1,0.649902,14,734,38,42014,0
2,1.623129,8,364,16,13186,0
3,1.681642,12,628,12,770,0
4,0.449454,10,534,6,268,0
...,...,...,...,...,...,...
257668,0.000005,2,104,0,0,0
257669,1.106101,20,18062,8,354,0
257670,0.000000,1,46,0,0,0
257671,0.000000,1,46,0,0,0


# Columns bzw. names anpassen

In [15]:
"""
df.reset_index(inplace=True)
del df['ts']
del df['uid']
del df['id.orig_h']
del df['id.orig_p']
del df['id.resp_h']
del df['id.resp_p']
del df['proto']
del df['service']
del df['tunnel_parents']
del df['history']
del df['local_orig']
del df['local_resp']
del df['conn_state']
del df['missed_bytes']
"""


"\ndf.reset_index(inplace=True)\ndel df['ts']\ndel df['uid']\ndel df['id.orig_h']\ndel df['id.orig_p']\ndel df['id.resp_h']\ndel df['id.resp_p']\ndel df['proto']\ndel df['service']\ndel df['tunnel_parents']\ndel df['history']\ndel df['local_orig']\ndel df['local_resp']\ndel df['conn_state']\ndel df['missed_bytes']\n"

In [18]:
#df.info()

In [19]:
#df_david = df[['duration','orig_pkts','orig_ip_bytes','resp_pkts','resp_ip_bytes','orig_bytes','resp_bytes']]

In [20]:
#df_david.reset_index(inplace=True)

In [17]:
#df_david

In [18]:
#del df_david['index']

In [21]:
#df_david

In [22]:
df_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 257673 entries, 0 to 257672
Data columns (total 6 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   duration       257673 non-null  float64
 1   orig_pkts      257673 non-null  int64  
 2   orig_ip_bytes  257673 non-null  int64  
 3   resp_pkts      257673 non-null  int64  
 4   resp_ip_bytes  257673 non-null  int64  
 5   label          257673 non-null  int64  
dtypes: float64(1), int64(5)
memory usage: 11.8 MB


In [21]:
#df_new.rename(columns={"dur": "duration"}, errors="raise", inplace=True)

In [23]:
df_new

,duration,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,label
0,0.121478,6,258,4,172,0
1,0.649902,14,734,38,42014,0
2,1.623129,8,364,16,13186,0
3,1.681642,12,628,12,770,0
4,0.449454,10,534,6,268,0
...,...,...,...,...,...,...
257668,0.000005,2,104,0,0,0
257669,1.106101,20,18062,8,354,0
257670,0.000000,1,46,0,0,0
257671,0.000000,1,46,0,0,0


In [23]:
#df_new = df_new[['duration','orig_pkts','orig_ip_bytes','resp_pkts','resp_ip_bytes','label']]
#df_new.to_pickle('unsw-small.pickle')

In [24]:
#df_test.proto.unique()

In [24]:
df_new

,duration,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,label
0,0.121478,6,258,4,172,0
1,0.649902,14,734,38,42014,0
2,1.623129,8,364,16,13186,0
3,1.681642,12,628,12,770,0
4,0.449454,10,534,6,268,0
...,...,...,...,...,...,...
257668,0.000005,2,104,0,0,0
257669,1.106101,20,18062,8,354,0
257670,0.000000,1,46,0,0,0
257671,0.000000,1,46,0,0,0


In [25]:
import joblib
#joblib.dump(rf, "./random_forest.joblib")
loaded_rf = joblib.load("../model/random_forest.joblib")

In [26]:
#del df_david['orig_bytes']
#del df_david['resp_bytes']

In [28]:
df_david.info()

In [29]:
df_david.to_pickle('davidsSet.pickle')

In [30]:
#df_david['duration'] = pd.to_numeric(df['duration'],errors='coerce')
#df_david['duration'] = df_david['duration'].astype(float)
count = 0
df_david['duration'] = [0.0 if isinstance(x,str) else x for x in df_david['duration']]



In [29]:
results = loaded_rf.predict(df_david)

In [30]:
df_david["prediction"] = 0.0

<ipython-input-30-cf133fa25b5a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_david["prediction"] = 0.0


In [31]:
df["prediction"] = 0.0

In [32]:
for i in range(len(results)):
    df["prediction"][i] = results[i] 

<ipython-input-32-9a547dc4d379>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["prediction"][i] = results[i]


In [33]:
df.to_pickle('connlog-with-prediction.pickle')

In [34]:
len(df_david)

475592

In [35]:
len(results)

475592

In [36]:
for i in range(len(results)):
    df_david["prediction"][i] = results[i] 

<ipython-input-36-e61857a9eb71>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_david["prediction"][i] = results[i]
/home/micha/anaconda3/envs/hackathon2/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3418: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


In [37]:
df_david.prediction.describe()

count    475592.000000
mean          0.142641
std           0.349707
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max           1.000000
Name: prediction, dtype: float64

In [38]:
df_david[df_david.prediction>=0.99]

,duration,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,prediction
7,209.875541,151036,2485906334,181125,9429656,1.0
10,26411.548384,0,0,2642,137576,1.0
13,22334.079740,0,0,10066,589936,1.0
118,232.883177,171676,2825660852,201907,10507844,1.0
167,6.543828,883,103540,98,2150104,1.0
...,...,...,...,...,...,...
475463,55.150433,11,615,4,160,1.0
475465,55.077840,11,615,4,160,1.0
475505,54.953765,11,615,5,200,1.0
475513,54.856970,10,551,3,120,1.0


In [40]:
import numpy as np
import pandas as pd
from pandas_profiling import ProfileReport

In [41]:
profile = ProfileReport(df_david, title='Prediction Profiling Report', minimal=True)

/home/micha/anaconda3/envs/hackathon2/lib/python3.8/site-packages/pandas/core/frame.py:4300: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [42]:
type(profile)

pandas_profiling.profile_report.ProfileReport

In [43]:
profile.to_file("../PredictionProfiling.html")

In [ ]:
#profile = ProfileReport(df_test, title='TestSet Profiling Report')

In [ ]:
#profile.to_file("ProfileTest.html")